# Using Open-source HuggingFace Models to Generate QAs from Raw Data in JSON format

In this example, we will show you how to generate question-answers (QAs) from given text strings using open-source Huggingface models via uniflow's [HuggingFaceModelFlow](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/model_flow.py#L86).

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

### Update system path

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Install libraries

In [2]:
!{sys.executable} -m pip install -q transformers accelerate bitsandbytes scipy

### Import dependency

In [3]:
from dotenv import load_dotenv
from IPython.display import display

from uniflow.flow.client import TransformClient
from uniflow.flow.config import HuggingfaceModelConfig, TransformQAHuggingFaceJsonFormatConfig
from uniflow.op.prompt import Context

load_dotenv()

/opt/conda/envs/uniflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

### Prepare sample prompts

First, we need to demonstrate sample prompts for LLM, those include instruction and sample json format. We do this by giving a sample instruction and list of `Context` examples to the `PromptTemplate` class. However, since we are using the default `PromptTemplate` in this example, we will not create it separately.

Second, we craft some dummy sample raw text strings. Below, we build a dataset with some text strings.

In [4]:
raw_context_input = [
    """We believe our success depends upon our capabilities in areas such as design, research and development, \
production and marketing and is supported and protected by our intellectual property rights, such as \
trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy \
of applying for and registering intellectual property rights in the United States and select foreign countries \
on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously \
protect our intellectual property, including trademarks, patents and trade secrets against third-party \
infringement and misappropriation.""",
    """In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) \
establishing the theory of information. In his article, Shannon introduced the concept of information entropy \
for the first time. We will begin our journey here.""",
    """The chain rule states that the derivative of a composite function (a function composed of another \
function) is equal to the derivative of the outer function multiplied by the derivative of the inner function.\
Mathematically, it can be written as: \(\frac{d}{dx}g(h(x)) = \frac{dg}{dh}(h(x))\cdot \frac{dh}{dx}(x)\).""",
    """Hypothesis testing involves making a claim about a population parameter based on sample data, and then \
conducting a test to determine whether this claim is supported or rejected. This typically involves \
calculating a test statistic, determining a significance level, and comparing the calculated value to a \
critical value to obtain a p-value. """
]

Next, for the given raw text strings `raw_context_input` above, we convert them to the `Context` class to be processed by `uniflow`.

In [5]:

input_data = [
    Context(context=data)
    for data in raw_context_input
]

print("sample size of processed input data: ", len(input_data))

input_data[:2]


sample size of processed input data:  4


[Context(context='We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously protect our intellectual property, including trademarks, patents and trade secrets against third-party infringement and misappropriation.'),
 Context(context='In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) establishing the theory of information. In his article, Shannon introduced the concept of information entropy for the first time. We will begin our journey here.')]

### Use LLM to generate data

In this example, we will use the [TransformQAHuggingFaceJsonFormatConfig](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/config.py#L128)'s default LLM to generate questions and answers. Let's import the config and client of this model.

Here in this example, we use our default `PromptTemplate` to the `TransformQAHuggingFaceJsonFormatConfig`, but you can use your customized instructions and examples instead if you want.

Note, base on your GPU memory, you can set your optimal `batch_size` below. (We attached our `batch_size` benchmarking results in the appendix of this notebook.)

In [6]:
config = TransformQAHuggingFaceJsonFormatConfig(
    model_config=HuggingfaceModelConfig(
            batch_size=1,
            response_start_key="question", response_format={"type": "json_object"}
        )
)
client = TransformClient(config)

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.51s/it]


In [7]:
print(config)

TransformQAHuggingFaceJsonFormatConfig(flow_name='TransformHuggingFaceFlow', model_config=HuggingfaceModelConfig(model_name='mistralai/Mistral-7B-Instruct-v0.2', model_server='HuggingfaceModelServer', batch_size=1, neuron=False, load_in_4bit=False, load_in_8bit=True, max_new_tokens=768, do_sample=False, temperature=0.0, num_beams=1, num_return_sequences=1, repetition_penalty=1.2, response_start_key='question', response_format={'type': 'json_object'}), num_thread=1, prompt_template=PromptTemplate(instruction='\n        Generate one question and its corresponding answer based on the last context in the last\n        example. Follow the format of the examples below to include context, question, and answer in the response.\n        ', few_shot_prompt=[Context(context='The quick brown fox jumps over the lazy black dog.', question='What is the color of the fox?', answer='brown.'), Context(context='The quick brown fox jumps over the lazy black dog.', question='What is the color of the dog?', 

Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.

In [8]:
output = client.run(input_data)

  0%|          | 0/4 [00:00<?, ?it/s]

/opt/conda/envs/uniflow/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 4/4 [00:47<00:00, 11.80s/it]


In [11]:
output

[{'output': [{'response': [{'context': 'We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously protect our intellectual property, including trademarks, patents and trade secrets against third-party infringement and misappropriation.',
      'question': 'What types of intellectual property does the company prioritize protecting?',
      'answer': 'The company prioritizes protecting intellectual property rights such as trademarks, utility and design patents, copyrights, and trade secrets.'}],
    'error': 'No errors.'}

### Process the output

Let's take a look of the generated output, which is already a list of JSON

In [12]:
from pprint import pprint

result = output[0]['output'][0]['response'][0] ## decode output

pprint(result, sort_dicts=False)

{'context': 'We believe our success depends upon our capabilities in areas '
            'such as design, research and development, production and '
            'marketing and is supported and protected by our intellectual '
            'property rights, such as trademarks, utility and design patents, '
            'copyrights, and trade secrets, among others. We have followed a '
            'policy of applying for and registering intellectual property '
            'rights in the United States and select foreign countries on '
            'trademarks, inventions, innovations and designs that we deem '
            'valuable. W e also continue to vigorously protect our '
            'intellectual property, including trademarks, patents and trade '
            'secrets against third-party infringement and misappropriation.',
 'question': 'What types of intellectual property does the company prioritize '
             'protecting?',
 'answer': 'The company prioritizes protecting intellect

Congrats! Your question answers from the given knowledge context are generated!


---


## Appendix

We benchmarked to see the optimal `batch_size` for the `TransformQAHuggingFaceJsonFormatConfig` flow. The answer is "It depends on your data token length, your GPU memory, your LLM size, etc." In the following experiment, we use an AWS `g5.2xlarge` instance that has a GPU with 24G memory and a quantized LLM (2G). We still use the above raw data strings `raw_context_input`.


Here are the results:

- batch_size = 1
    100%|██████████| 4/4 [00:47<00:00, 11.80s/it]
- batch_size = 2
    100%|██████████| 2/2 [00:35<00:00, 17.87s/it]
- batch_size = 4
    100%|██████████| 1/1 [00:13<00:00, 13.34s/it]
- batch_size = 8
    100%|██████████| 1/1 [00:13<00:00, 13.20s/it]
- batch_size = 16
    100%|██████████| 1/1 [00:13<00:00, 13.48s/it]
- batch_size = 32
    100%|██████████| 1/1 [00:13<00:00, 13.36s/it]
- batch_size = 64
    100%|██████████| 1/1 [00:13<00:00, 13.28s/it]
- batch_size = 128:
    100%|██████████| 1/1 [00:13<00:00, 13.33s/it]
- batch_size = 256:
    100%|██████████| 1/1 [00:13<00:00, 13.33s/it]

As you can see, the processing time is much shorter if `batch_size=4` compared with `batch_size=1`. However, the increase in the speed will become more prominent when the input is bigger. While, it might lead to OOM error if `batch_size` is too large when handling some big dataset.

## End of the notebook

Check more Uniflow use cases in the [example folder](https://github.com/CambioML/uniflow/tree/main/example/model#examples)!

<a href="https://www.cambioml.com/" title="Title">
    <img src="../image/cambioml_logo_large.png" style="height: 100px; display: block; margin-left: auto; margin-right: auto;"/>
</a>